# Dependency Injection
> Because everything is better once it is over-engineered.

In [ ]:
# default_exp di

In [ ]:
# hide
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


In [ ]:
# exports
import dataclasses
from dataclasses import dataclass

In [ ]:
# exports


@dataclass
class DependencyInjection:
    config: dict
    supported_types: list = None

    def create_dataclass_type(self, dataclass_type, *args, **kwargs):
        resolved_args = self.resolve_dataclass_args(dataclass_type)
        final_args = {**resolved_args, **kwargs}

        args_str = ','.join(list(args) + [f'{key}={value}' for key, value in final_args.items()])
        print(f"Creating: {dataclass_type.__qualname__}({args_str})")
        return dataclass_type(*args, **final_args)

    def resolve_dataclass_args(self, dataclass_type):
        fields = dataclasses.fields(dataclass_type)

        resolved_args = {}
        field: dataclasses.Field
        for field in fields:
            type_specific_name = f"{dataclass_type.__qualname__}__{field.name}"
            if type_specific_name in self.config:
                resolved_args[field.name] = self.config[type_specific_name]
            elif field.name in self.config:
                resolved_args[field.name] = self.config[field.name]

            if self.supported_types and field.type in self.supported_types:
                resolved_args[field.name] = self.create_dataclass_type(field.type)

        print(f"Resolved: {dataclass_type.__qualname__} with {resolved_args}")

        return resolved_args

In [ ]:
@dataclass
class SimpleType:
    a_config_variable: int


di = DependencyInjection(dict(a_config_variable=5))

di.create_dataclass_type(SimpleType)

Resolved: SimpleType with {'a_config_variable': 5}
Creating: SimpleType(a_config_variable=5)


SimpleType(a_config_variable=5)

In [ ]:
@dataclass
class SimpleTypeA:
    a_config_variable: int


@dataclass
class SimpleTypeB:
    a_config_variable: int


di = DependencyInjection(dict(a_config_variable=5, SimpleTypeB__a_config_variable=10))

di.create_dataclass_type(SimpleTypeA), di.create_dataclass_type(SimpleTypeB)

Resolved: SimpleTypeA with {'a_config_variable': 5}
Creating: SimpleTypeA(a_config_variable=5)
Resolved: SimpleTypeB with {'a_config_variable': 10}
Creating: SimpleTypeB(a_config_variable=10)


(SimpleTypeA(a_config_variable=5), SimpleTypeB(a_config_variable=10))

In [ ]:
@dataclass
class ComplexTypeC:
    simple_type_b: SimpleTypeB


di = DependencyInjection(dict(a_config_variable=5, SimpleTypeB__a_config_variable=10), [SimpleTypeB])

di.create_dataclass_type(ComplexTypeC)

Resolved: SimpleTypeB with {'a_config_variable': 10}
Creating: SimpleTypeB(a_config_variable=10)
Resolved: ComplexTypeC with {'simple_type_b': SimpleTypeB(a_config_variable=10)}
Creating: ComplexTypeC(simple_type_b=SimpleTypeB(a_config_variable=10))


ComplexTypeC(simple_type_b=SimpleTypeB(a_config_variable=10))